In [1]:
from pyspark.sql import SparkSession
db_path="s3a://go01-demo/data/warehouse/tablespace/managed/hive"
spark = (SparkSession
    .builder
    .appName("test-spark")
    .config("spark.sql.warehouse.dir", db_path)
    .config("spark.hadoop.fs.s2a.s3guard.ddb.region", "us-east-1")
    .config("'spark.kerberos.access.hadoopFileSystems","s3a://go01-demo/")
    .master("local[5]") # should be possible to change this to SPARK on Yarn or SPARK on Kubernetes
    .getOrCreate())

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark.sql("select event_timestamp, driver_id, conv_rate, acc_rate, avg_daily_trips, created from default.driver_stats where driver_id=1001").show()

+-------------------+---------+-----------+-----------+---------------+--------------------+
|    event_timestamp|driver_id|  conv_rate|   acc_rate|avg_daily_trips|             created|
+-------------------+---------+-----------+-----------+---------------+--------------------+
|2022-12-06 08:00:00|     1001| 0.95960015| 0.31500015|            706|2022-12-21 08:33:...|
|2022-12-06 09:00:00|     1001| 0.21780412|0.062166434|            232|2022-12-21 08:33:...|
|2022-12-06 10:00:00|     1001|  0.7174978|0.045171406|            925|2022-12-21 08:33:...|
|2022-12-06 11:00:00|     1001|  0.7511095| 0.11798655|            879|2022-12-21 08:33:...|
|2022-12-06 12:00:00|     1001|  0.5841053|  0.7301582|            796|2022-12-21 08:33:...|
|2022-12-06 13:00:00|     1001|  0.5143138|  0.5857045|            587|2022-12-21 08:33:...|
|2022-12-06 14:00:00|     1001|  0.7667928| 0.44835535|            438|2022-12-21 08:33:...|
|2022-12-06 15:00:00|     1001| 0.77055275| 0.17851345|            946

In [13]:
parDF=spark.read.parquet("s3a://go01-demo/user/vishr/driver_hourly_stats.parquet")

In [14]:
parDF.show()

+-------------------+---------+------------+-----------+---------------+--------------------+
|    event_timestamp|driver_id|   conv_rate|   acc_rate|avg_daily_trips|             created|
+-------------------+---------+------------+-----------+---------------+--------------------+
|2022-11-13 11:00:00|     1003|  0.15853529|  0.1788818|             18|2022-11-28 11:15:...|
|2022-11-13 12:00:00|     1003|  0.20408256|  0.7754877|            185|2022-11-28 11:15:...|
|2022-11-13 13:00:00|     1003|  0.44259834| 0.33490667|            927|2022-11-28 11:15:...|
|2022-11-13 14:00:00|     1003| 0.016885195| 0.10977515|            898|2022-11-28 11:15:...|
|2022-11-13 15:00:00|     1003|  0.24754624|  0.2591652|            904|2022-11-28 11:15:...|
|2022-11-13 16:00:00|     1003|  0.36790988|  0.5196834|            215|2022-11-28 11:15:...|
|2022-11-13 17:00:00|     1003|  0.74756473| 0.06004086|            450|2022-11-28 11:15:...|
|2022-11-13 18:00:00|     1003|  0.89914817|  0.9461962|    

In [5]:
parDF.write \
    .format("parquet") \
    .saveAsTable('driver_stats')

In [7]:
spark.sql("select * from driver_stats").show()

+-------------------+---------+------------+----------+---------------+--------------------+
|    event_timestamp|driver_id|   conv_rate|  acc_rate|avg_daily_trips|             created|
+-------------------+---------+------------+----------+---------------+--------------------+
|2022-11-11 08:00:00|     1005|3.6625357E-4|0.22111285|            805|2022-11-26 08:09:...|
|2022-11-11 09:00:00|     1005|  0.71298265|0.60847104|            122|2022-11-26 08:09:...|
|2022-11-11 10:00:00|     1005|  0.34247807|0.59768647|            777|2022-11-26 08:09:...|
|2022-11-11 11:00:00|     1005|  0.06899766|0.65062225|            800|2022-11-26 08:09:...|
|2022-11-11 12:00:00|     1005|   0.5121142| 0.8556645|            909|2022-11-26 08:09:...|
|2022-11-11 13:00:00|     1005|  0.18383828|0.32264683|            351|2022-11-26 08:09:...|
|2022-11-11 14:00:00|     1005|   0.3206321| 0.8428178|            349|2022-11-26 08:09:...|
|2022-11-11 15:00:00|     1005|   0.4921242|0.82953787|            708

In [10]:
spark.sql("DESCRIBE FORMATTED default.driver_stats").show()

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|     event_timestamp|           timestamp|   null|
|           driver_id|              bigint|   null|
|           conv_rate|               float|   null|
|            acc_rate|               float|   null|
|     avg_daily_trips|                 int|   null|
|             created|           timestamp|   null|
|                    |                    |       |
|# Detailed Table ...|                    |       |
|            Database|             default|       |
|               Table|        driver_stats|       |
|               Owner|     vishrajagopalan|       |
|        Created Time|Wed Dec 21 08:50:...|       |
|         Last Access|             UNKNOWN|       |
|          Created By|Spark 3.2.0.1.15....|       |
|                Type|            EXTERNAL|       |
|            Provider|             parquet|       |
|    Table P